# Audio Extraction

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
print(f'\tOPENAI_API_KEY={os.getenv("OPENAI_API_KEY")[:20]}')

	OPENAI_API_KEY=sk-proj-iKU13YeoxNgF


## ffmpeg 사용: mp4 -> mp3 오디오 추출

In [2]:
!ffmpeg -i D:\Lang2505\dataset\podcast.mp4 -vn D:\Lang2505\dataset\audio.mp3 -y

ffmpeg version 2025-07-28-git-dc8e753f32-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.1.0 (Rev6, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-liboapv --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjx

In [ ]:
# -i 옵션 : input. 입력 파일 지정 (여러개 지정 가능)
# -vn 옵션 : disable video. 비디오는 무시하고 오디오만 추출
# -y 옵션 : 덮어쓰기 할때 y/n 사용자 입력 대디하지 않고 yes 로 진행

## 오디오 추출 함수 작성

In [3]:
import subprocess

In [4]:
base_path = r'D:\Lang2505\dataset'  # 동영상 경로
out_path = r'D:\Lang2505\dataset\out' # 분할 오디오 추출 경로

video_file = 'podcast.mp4'
audio_file = 'audio.mp3'

src_path = os.path.join(base_path, video_file)
dst_path = os.path.join(base_path, audio_file)

print(src_path)
print(dst_path)

D:\Lang2505\dataset\podcast.mp4
D:\Lang2505\dataset\audio.mp3


In [5]:
# video_path: 동영상 경로
# audio_path: 추출한 오디오 경로
def extract_audio_from_video(video_path, audio_path):
    # CLI 에서 실행할 command 준비
    # 커맨드창 (CLI) 에서 명령은 아래와 같다.
    #  > ffmpeg -i D:\dataset\podcast.mp4 -vn D:\NLP2501\dataset\audio.mp3 -y
    command = ["ffmpeg", "-i", video_path, "-vn", audio_path, "-y"]  
    subprocess.run(command)
    

In [6]:
extract_audio_from_video(src_path, dst_path)

# Cutting the Audio

In [ ]:
# Whisper API 의 최대 입력 오디오길이가 10분.
# 원돈 오디오를 10분 단위로 쪼개기

In [ ]:
# pydub 패키지
#  공식: https://github.com/jiaaro/pydub 
#  pip install pydub  <- 설치 필요  (사전에 ffmpeg 가 설치되고 경로 설정도 되어 있어야 한다)


In [7]:
from pydub import AudioSegment

In [8]:
track = AudioSegment.from_mp3(dst_path)

In [20]:
# track

In [10]:
track.duration_seconds

4422.426122448979

In [11]:
len(track) # ms

4422426

In [19]:
# 오디오의 첫 5분선택
five_minutes = 5 * 60 * 1000 # 단위 ms

first_five = track[:five_minutes]

# first_five

In [13]:
first_five.duration_seconds

300.0

In [14]:
# export 하기
first_five.export(os.path.join(out_path, 'first_five.mp3'), format='mp3')

<_io.BufferedRandom name='D:\\Lang2505\\dataset\\out\\first_five.mp3'>

## 오디오 파일 분할 

In [15]:
ten_minutes = 10 * 60 * 1000

In [16]:
len(track) / ten_minutes

7.37071

In [17]:
import math

chunks = math.ceil(len(track) / ten_minutes)

chunks  # 쪼개질 덩어리들.

8

In [21]:
for i in range(chunks):
    start_time = i * ten_minutes
    end_time = (i + 1) * ten_minutes
    print(f"{i} start: {start_time}  end {end_time}")    

0 start: 0  end 600000
1 start: 600000  end 1200000
2 start: 1200000  end 1800000
3 start: 1800000  end 2400000
4 start: 2400000  end 3000000
5 start: 3000000  end 3600000
6 start: 3600000  end 4200000
7 start: 4200000  end 4800000


In [22]:
for i in range(chunks):
    start_time = i * ten_minutes
    end_time = (i + 1) * ten_minutes
    print(f"{i} start: {start_time}  end {end_time}")  

    chunk = track[start_time:end_time]
    chunk.export(os.path.join(out_path, f'chunk_{i}.mp3'), format='mp3')

0 start: 0  end 600000
1 start: 600000  end 1200000
2 start: 1200000  end 1800000
3 start: 1800000  end 2400000
4 start: 2400000  end 3000000
5 start: 3000000  end 3600000
6 start: 3600000  end 4200000
7 start: 4200000  end 4800000


In [23]:
!dir D:\Lang2505\dataset\out\*.mp3

 D 드라이브의 볼륨: 새 볼륨
 볼륨 일련 번호: 02B1-19A2

 D:\Lang2505\dataset\out 디렉터리

2025-07-30  오후 08:02         9,600,774 chunk_0.mp3
2025-07-30  오후 08:02         9,600,774 chunk_1.mp3
2025-07-30  오후 08:03         9,600,774 chunk_2.mp3
2025-07-30  오후 08:03         9,600,774 chunk_3.mp3
2025-07-30  오후 08:03         9,600,774 chunk_4.mp3
2025-07-30  오후 08:03         9,600,774 chunk_5.mp3
2025-07-30  오후 08:03         9,600,774 chunk_6.mp3
2025-07-30  오후 08:03         3,559,592 chunk_7.mp3
2025-07-30  오후 07:45         4,800,931 first_five.mp3
               9개 파일          75,565,941 바이트
               0개 디렉터리  461,113,454,592 바이트 남음


# Whisper Transcript

In [24]:
import openai

In [27]:
transcript = openai.audio.transcriptions.create(
    model='whisper-1',
    file=open(os.path.join(out_path, 'chunk_0.mp3'), 'rb'),
    language='en',
)

transcript

Transcription(text="If success is this lagging indicator of commitment now, how can you be sure that you are paying your dues? The best-selling author and host. The number one health and wellness podcast. On Purpose with Jay Shetty. Society has gone in the direction of becoming addicted to pleasure. Yes. Or pleasure-seeking. Where, from the Stoics' perspective, why did we even ever go down that road? Like, why did we leave wisdom and self-control? Or did we never have it at all and we've always been trying to balance it? Yeah. I mean, I guess that's the big question is, like, why do we take something that we like too far? Yeah. Right. So the Epicureans would say like, look, drinking is great, but if you have a hangover the next day, was it actually so great? And so, you know, if you, if you push the pleasure too far, it becomes not pleasurable, but in the moment that feels very far away, right? Like in the moment you want the thing now, obviously sex is this thing for people, it's like

In [29]:
# first_five

In [30]:
transcript.text

"If success is this lagging indicator of commitment now, how can you be sure that you are paying your dues? The best-selling author and host. The number one health and wellness podcast. On Purpose with Jay Shetty. Society has gone in the direction of becoming addicted to pleasure. Yes. Or pleasure-seeking. Where, from the Stoics' perspective, why did we even ever go down that road? Like, why did we leave wisdom and self-control? Or did we never have it at all and we've always been trying to balance it? Yeah. I mean, I guess that's the big question is, like, why do we take something that we like too far? Yeah. Right. So the Epicureans would say like, look, drinking is great, but if you have a hangover the next day, was it actually so great? And so, you know, if you, if you push the pleasure too far, it becomes not pleasurable, but in the moment that feels very far away, right? Like in the moment you want the thing now, obviously sex is this thing for people, it's like the thing you're a

In [31]:
import openai
import glob

def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(os.path.join(chunk_folder, "chunk*.mp3"))
    for file in files:
        with open(file, "rb") as audio_file, open(destination, "a") as text_file:  # append mode 로 
            print(file, '녹취록 가져오는중...', end='')
            # 각 chunk 별로 녹취록 작성.
            transcript = openai.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file,
                language="en"
            )
            
            text_file.write(transcript.text)  # 곧바로 텍스트 파일에 저장
            print('완료')

transcribe_chunks(out_path, os.path.join(out_path, 'transcript.txt'))

D:\Lang2505\dataset\out\chunk_0.mp3 녹취록 가져오는중...완료
D:\Lang2505\dataset\out\chunk_1.mp3 녹취록 가져오는중...완료


# glob 관련 이슈 

In [32]:
glob.glob(r'D:\Lang2505\dataset\out\*.mp3')

['D:\\Lang2505\\dataset\\out\\chunk_0.mp3',
 'D:\\Lang2505\\dataset\\out\\chunk_1.mp3',
 'D:\\Lang2505\\dataset\\out\\first_five.mp3']

In [ ]:
# glob 은 list 에 정렬되는 순서가 파일이름으로 보장되지 않는다!